# 准备每天数据

author：BOB

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import GetJsonData
data = GetJsonData.getData()

In [3]:
data.keys()

dict_keys(['city', 'code', 'msg', 'province'])

In [4]:
len(data['city'])

8

In [5]:
folderName = 'historyData/' #存放历史数据文件夹 
if not os.path.exists(folderName):
    os.makedirs(folderName)

In [6]:
def rate(x):
    '''
    func: 返回感染率
    return: float
    '''
    if 0 <= x <= 9:
        return 0.4
    elif 10 <= x <= 99:
        return 0.5
    elif 100 <= x <= 299:
        return 0.6
    elif 300 <= x <= 499:
        return 0.7
    elif 500 <= x <= 699:
        return 0.8
    elif 700 <= x <= 899:
        return 0.9
    
    else:
        return 1.0
    '''
    elif 900 <= x <= 1999:
        return 1.0
    
    
    elif 2000 <= x <= 2999:
        return 0.8
    elif 3000 <= x <= 4999:
        return 0.9
    '''

In [7]:
for PerDayData in data['city'][:]:
    fileName = PerDayData['Time'] + '.csv' #文件名
    filePath = os.path.join(folderName, fileName) #保存 csv 路径
    
    #构成字典 
    dataList = {'cityName': PerDayData['City'], 
                'confirmed': PerDayData['NcovNum']}
    DF_data = pd.DataFrame(dataList, columns=['cityName', 'confirmed'])
    DF_data['infectionRate'] = DF_data['confirmed']
    DF_data['infectionRate'] = DF_data['infectionRate'].apply(lambda x: rate(x))
    DF_data.sort_values('infectionRate',inplace=True, ascending=False)
    DF_data.to_csv(filePath, index=False)
    #print(set(DF_data['infectionRate']))
    

# 提取每天数据 添加经纬度

In [8]:
filePath_List = os.listdir(folderName) #获取当前历史数据下所有文件名
filePath_List.sort()
data = [] #总数据列表
filePath_List

['2020-01-23.csv',
 '2020-01-24.csv',
 '2020-01-25.csv',
 '2020-01-26.csv',
 '2020-01-27.csv',
 '2020-01-28.csv',
 '2020-01-29.csv',
 '2020-01-30.csv']

In [10]:
#获取经纬度 DF
cityCoordinate = pd.read_csv('City_Coorinary.csv')
cityCoordinate.head()

,cityName,longitude,latitude
0,武汉,114.311582,30.598467
1,黄冈,114.878490,30.459359
2,孝感,113.922510,30.930689
3,襄阳,112.128537,32.014797
4,荆州,112.245523,30.340842


In [11]:
#filepath = os.path.join(folderName, filePath_List[-1])
for path in filePath_List: #遍历每一个文件名
    filepath = os.path.join(folderName, path) #文件路径
    DayData = pd.read_csv(filepath)
    print(path)
    #融合经纬度
    UpdateData = pd.merge(DayData, cityCoordinate, on='cityName', how = 'left')
    UpdateData = UpdateData.dropna(subset=["longitude","latitude"])
    #获取 经纬度, 感染率数据
    #UpdateData['infectionRate'] = 1
    PerData = UpdateData[['latitude', 'longitude', 'infectionRate']].values
    PerData = PerData.tolist()
    data.append(PerData)

2020-01-23.csv
2020-01-24.csv
2020-01-25.csv
2020-01-26.csv
2020-01-27.csv
2020-01-28.csv
2020-01-29.csv
2020-01-30.csv


# data 就是总数据 可视化加到下面

In [12]:
import folium
import folium.plugins as plugins

In [13]:
from datetime import datetime, timedelta
time_index = []
time_index = [
    (datetime(2020, 1, 23) + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(data))
]
time_index

['2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30']

In [14]:
data[0]

[[30.59846674, 114.31158159999998, 0.7],
 [22.54845664, 114.06455179999999, 0.5],
 [28.66219406, 121.4274347, 0.5],
 [30.45935886, 114.8784905, 0.5],
 [30.93068923, 113.9225101, 0.5],
 [32.26127087, 118.3394061, 0.4],
 [33.8506427, 115.7844632, 0.4],
 [30.53095657, 117.0636039, 0.4],
 [30.67088379, 117.498421, 0.4],
 [32.921523699999995, 117.39551329999999, 0.4],
 [37.87698903, 112.5563915, 0.4],
 [43.8219535, 125.3306021, 0.4],
 [31.74145082, 116.5264097, 0.4],
 [43.84356783, 126.5556345, 0.4],
 [45.14740419, 124.83148190000001, 0.4],
 [46.66003218, 126.97535690000001, 0.4],
 [46.59363318, 125.10865759999999, 0.4],
 [24.33196139, 109.4344219, 0.4],
 [32.89606099, 115.82043610000001, 0.4],
 [31.82657783, 117.23344270000001, 0.4],
 [24.87399815, 102.8524484, 0.4],
 [35.11067124, 118.363533, 0.4],
 [25.24288572, 110.20354540000001, 0.4],
 [37.51643055, 122.127541, 0.4],
 [36.71265155, 119.16837790000001, 0.4],
 [36.072227500000004, 120.38945520000001, 0.4],
 [27.7032086, 112.0015035, 0.4

In [15]:
m = folium.Map([30.52, 114.31], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m

In [16]:
m = folium.Map([30.52, 114.31], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m

In [17]:
for i in data:
    print(np.array(i).shape)

(107, 3)
(158, 3)
(204, 3)
(232, 3)
(253, 3)
(268, 3)
(281, 3)
(261, 3)
